In [19]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Calculate lean angle of a pole using code execution

This notebook uses the Gemini model to analyze lean angle of a pole. It fetches image URIs from a BigQuery table and then uses prompts to instruct the Gemini model to perform the following analyses:
- Detect the pole
- Run canny Edge detection model
- Run Hough line detection algorithm
- Output the angle as an integer



## Install required libraries

In [ ]:
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-genai

## Configuration

In [ ]:
PROJECT_ID = ''  # @param {type:"string"}
REGION = ''      # @param {type:"string"}
MODEL_ID = "gemini-2.5-flash" # @param {type:"string"}
BIGQUERY_DATASET_ID = '' # @param {type:"string"}
BIGQUERY_TABLE = "latest_observations" # @param {type:"string"}
ASSET_TYPE = "ASSET_CLASS_UTILITY_POLE" # @param {type:"string"}
LIMIT = 5 # @param {type:"integer"}

## Imports and Vertex AI Initialization

In [22]:
import vertexai
from google.cloud import bigquery
from google import genai
from google.genai.types import (
    GenerateContentConfig,
    HarmBlockThreshold,
    HarmCategory,
    HttpOptions,
    Part,
    Tool,
    ToolCodeExecution,
    SafetySetting,
)
import pandas as pd
import re
import json



## Initialize the vertex AI SDK

In [23]:
vertexai.init(project=PROJECT_ID, location=REGION)
print("pandas, re, and json libraries have been successfully imported and Vertex AI initialized.")

pandas, re, and json libraries have been successfully imported and Vertex AI initialized.


## Fetch Image URIs from BigQuery

In [ ]:
BIGQUERY_SQL_QUERY = f"""
SELECT
  *
FROM
  `{PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_TABLE}`
   WHERE asset_type = "{ASSET_TYPE}"
LIMIT {LIMIT};
"""

try:
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    query_job = bigquery_client.query(BIGQUERY_SQL_QUERY)
    query_response_data = [dict(row) for row in query_job]
    gcs_uris = [item.get("gcs_uri") for item in query_response_data if item.get("gcs_uri")]
    print(f"Successfully fetched {len(gcs_uris)} GCS URIs:")
    for uri in gcs_uris:
        print(uri)
except Exception as e:
    print(f"An error occurred while querying BigQuery: {e}")

## Define Analysis Prompts

In [25]:
PROMPT = """You will be provided with an image for analysis.

Your task is to perform Canny edge detection and Hough angle calculations to determine the angle of a pole in the image. Follow these steps exactly:

1.  Load the Image: Access the provided image file.
2.  Convert to Grayscale: Convert the color image into a single-channel grayscale image.
3.  Apply Gaussian Blur: Apply a Gaussian blur to smooth the image and reduce noise.
4.  Filter for Vertical/Near-Vertical Lines: Identify lines that are likely to represent a pole. These are typically near-vertical lines.
5.  Perform Canny Edge Detection: Apply the Canny edge detection algorithm to find edges in the image. Use appropriate thresholds.
6.  Perform Hough Line Transform: Apply the Probabilistic Hough Line Transform to detect straight lines in the Canny edge-detected image.
7.  Calculate Pole Angle: From the detected pole lines, calculate the average angle of the pole relative to the vertical axis.
8.  Format the Output: Your final output must be ONLY the calculated angle, rounded to the nearest whole number (integer). Do not include any other text, explanation, or code."""

## Create vertex client

In [26]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

## Run Analysis

In [27]:
# Initialize the genai Client and the code execution Tool

code_execution_tool = Tool(code_execution=ToolCodeExecution())

def analyze_image(gcs_uri: str, prompt: str) -> str:
    """Analyzes an image from GCS using the Gemini model with code execution, returning None on error."""
    try:
        contents = [
            prompt,
            Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})
        ]
        # Update the generate_content call to include the code execution tool
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=contents,
            config=GenerateContentConfig(
            tools=[code_execution_tool],
            temperature=0,
              )
        )
        print("# Code:")
        print(response.executable_code)
        print("# Outcome:")
        print(response.code_execution_result)
        return response.text
    except Exception as e:
        print(f"Error analyzing '{gcs_uri}': {e}")
        return None

results = []

if 'query_response_data' in locals() and query_response_data:
    for item in query_response_data:
        asset_id = item.get("asset_id")
        uri = item.get("gcs_uri")

        if not uri or not asset_id:
            continue

        print(f"--- Analyzing {uri} ---")

        # Analyze image with the new prompt for pole angle
        pole_angle_response = analyze_image(uri, PROMPT)

        # Parse pole angle result
        pole_angle = None
        try:
            if pole_angle_response:
                pole_angle = int(float(re.search(r'[-+]?\d*\.?\d+', pole_angle_response).group()))
        except (ValueError, AttributeError, TypeError):
            pole_angle = None

        # Store results
        result_item = {
            "asset_id": asset_id,
            "pole_angle": pole_angle
        }
        results.append(result_item)

    # Create and display DataFrame
    if results:
        df = pd.DataFrame(results)
        df.set_index('asset_id', inplace=True)
        print("\n--- Analysis Results ---")
        display(df)
    else:
        print("No data to display.")

else:
    print("No GCS URIs were found to analyze.")

--- Analyzing gs://geoai_published_ba3433e0-d709-4622-9577-66ab23587729__us/e8761cd3-c872-43cd-90aa-3ee99378090f/v0/o1:566314da74215735b48631b99f3b8429:00250082.jpg ---


# Code:
import cv2
import numpy as np
import math

# Image file name
image_path = "f_geoai_published_ba3433e0_d709_4622_9577_66ab23587729__us_e8761cd3_c872_43cd_90aa_3ee99378090f_v0_o1_566314da74215735b48631b99f3b8429_00250082.jpg"

# 1. Load the Image
image = cv2.imread(image_path)

if image is None:
    raise FileNotFoundError(f"Could not load image at {image_path}")

# 2. Convert to Grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 3. Apply Gaussian Blur
# Kernel size (5, 5) and standard deviation 0
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# 5. Perform Canny Edge Detection
# Adjust thresholds based on image characteristics.
# Lower threshold for weak edges, higher for strong edges.
# A common ratio is 1:2 or 1:3.
low_threshold = 50
high_threshold = 150
edges = cv2.Canny(blurred_image, low_threshold, high_threshold)

# 6. Perform Hough Line Transform
# Parameters for Probabilistic Hough Line Transform
# rho: Distance resolution of the accumulator in pi

# Code:
import cv2
import numpy as np
import math

# 1. Load the Image
image_path = "f_geoai_published_ba3433e0_d709_4622_9577_66ab23587729__us_e8761cd3_c872_43cd_90aa_3ee99378090f_v0_o1_01cdba18fbed09262fba4f92e67ddb00_00250082.jpg"
image = cv2.imread(image_path)

if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# 2. Convert to Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 3. Apply Gaussian Blur
# Kernel size (5, 5) and standard deviation 0 are common starting points.
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# 4. Perform Canny Edge Detection
# Adjust thresholds as needed. Lower threshold for weak edges, higher for strong edges.
# A common heuristic is to use a 1:2 or 1:3 ratio for low:high thresholds.
edges = cv2.Canny(blurred, 50, 150)

# 5. Perform Hough Line Transform
# Parameters for HoughLinesP:
# rho: Distance resolution of the accumulator in pixels.
# theta: Angle resolution of the accumulator in radians.
# threshold: Accumula

# Code:
import cv2
import numpy as np
import math

# 1. Load the Image
image_path = "f_geoai_published_ba3433e0_d709_4622_9577_66ab23587729__us_e8761cd3_c872_43cd_90aa_3ee99378090f_v0_o1_b83b2016a26a802b1ccb71c27753d65c_00250082.jpg"
image = cv2.imread(image_path)

if image is None:
    raise FileNotFoundError(f"Could not load image at {image_path}")

# 2. Convert to Grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 3. Apply Gaussian Blur
# Kernel size (5, 5) and standard deviation 0 are common starting points.
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# 5. Perform Canny Edge Detection
# Adjust thresholds as needed. Lower threshold for weak edges, higher for strong edges.
# A common ratio for Canny thresholds is 1:2 or 1:3.
edges = cv2.Canny(blurred_image, 50, 150)

# 6. Perform Hough Line Transform
# Parameters:
# image: Output of the Canny detector.
# rho: Distance resolution of the accumulator in pixels.
# theta: Angle resolution of the accumulator in 

# Code:
import cv2
import numpy as np
import math

# 1. Load the Image
image_path = "f_geoai_published_ba3433e0_d709_4622_9577_66ab23587729__us_e8761cd3_c872_43cd_90aa_3ee99378090f_v0_o1_cdc94564ab743089849f02decc3970f3_00250082.jpg"
image = cv2.imread(image_path)

if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# 2. Convert to Grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 3. Apply Gaussian Blur
# Kernel size (5,5) and sigmaX=0 are common starting points.
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# 5. Perform Canny Edge Detection
# Adjust thresholds as needed. Lower threshold for weak edges, higher for strong.
# A common heuristic is to use a 1:2 or 1:3 ratio for low:high thresholds.
# Let's try 50 and 150.
edges = cv2.Canny(blurred_image, 50, 150)

# 6. Perform Probabilistic Hough Line Transform
# Parameters:
# edges: Output of the Canny detector.
# rho: Distance resolution of the accumulator in pixels.
# theta: Ang

# Code:
import cv2
import numpy as np

# Load the image
image_path = "f_geoai_published_ba3433e0_d709_4622_9577_66ab23587729__us_e8761cd3_c872_43cd_90aa_3ee99378090f_v0_o1_80870297e7eb2fcc52aef93c52364d6c_00250082.jpg"
image = cv2.imread(image_path)

# Check if image loaded successfully
if image is None:
    raise FileNotFoundError(f"Could not load image from {image_path}")

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Perform Canny edge detection
# Adjusting thresholds based on common practices and potential image characteristics
edges = cv2.Canny(blurred, 50, 150)

# Perform Probabilistic Hough Line Transform
# Parameters: image, rho, theta, threshold, minLineLength, maxLineGap
# rho: Distance resolution of the accumulator in pixels.
# theta: Angle resolution of the accumulator in radians.
# threshold: Accumulator threshold parameter. Only lines that get enough votes (>threshold) are return

,pole_angle
asset_id,
t1:3bb60629975471a4966e3bd59aed5f31:ffff01ee,5
t1:6a21e109b0fc28eda5d7887a9686d922:ffff01ee,1
t1:000c345da88196b1502312ade92f5ca8:ffff01ee,4
t1:917054809f5e4de720fa89cb771fe202:ffff01ee,6
t1:cf1461018d092a5ac91876364629fd62:ffff01ee,6
